In [18]:
type Parser a = String -> [(a,String)]

In [19]:
satisfy :: (Char -> Bool) -> Parser Char
satisfy p (c:cs) | p c = [(c,cs)] 
satisfy _  _           = []

In [20]:
anyChar = satisfy (const True) -- statisfy (\_ -> True)
char x = satisfy (x==)
oneOf xs = satisfy (`elem` xs)

In [21]:
import Data.Char
space = satisfy isSpace
alphaNum = satisfy isAlphaNum
letter = satisfy isAlpha
lower = satisfy isLower
upper = satisfy isUpper
digit = satisfy isDigit

아래의 실행예처럼 주어진 문자열과 정확히 일치하는 만큼을 분석하여 처리하는 `string` 함수를 작성하라. (다음 과제의 첫 문제가 될 내용이다)
```haskell
string "abc" "abcdef" == [("abc","def")]
```

In [22]:
string :: String -> Parser String 
string ""     cs                 = [("",cs)]
string (x:xs) (c:cs) | undefined = undefined
string _      _                  = []

----

다음 과제의 두번째 문제는 아래 모든 개수를 다 고려하는 문법분석 함수 many1과 many를 가장 길게 매칭되는 경우만 결과로 하는 것으로 수정하라는 것

In [23]:
-- parse one or more times
many1 :: Parser a -> Parser [a]
many1 p cs = [([x], cs') | (x,cs')<-ps] -- 한번 처리한 결과 x를 리스트로
            ++
             [(x:xs, cs'') | (x,cs')<-ps, (xs,cs'')<-many1 p cs']
           where ps = p cs

-- parse zero or more time
many :: Parser a -> Parser [a]
many p cs = [([],cs)] ++ many1 p cs

In [24]:
spaces = many space

## 십진수 양의 정수를 분석하기

In [25]:
-- 맨 앞에는 0이 아닌 숫자, 그 다음에는 아무 십진수 숫자나

firstDigit = oneOf ['1'..'9'] -- 맨 앞글자를 처리 (딱 1개 글자)
digits = many digit       -- 뒷글자들을 처리 (0개 이상 글자)

In [26]:
(>>=) :: Parser a -> (a -> Parser b) -> Parser b
p >>= f = \cs -> [(y,cs'') | (x, cs')<-p cs, (y,cs'') <- f x cs']

infixr 1 >>=

In [27]:
return :: b -> Parser b
return v = \cs -> [(v,cs)] 

In [28]:
posint :: Parser Int
posint = firstDigit >>= \x  ->
         digits     >>= \xs ->
         return (read(x:xs))

# 자연수 Parser와  정수 Parser

이제 자연수 Parser와 정수 Parser를 만들어 보자. 자연수는 양의 정수이거나 0이며 정수는
자연수이거나 음의 정수이므로 이들 각각에 해당하는 Parser를 만들어 결과가 그 중 어느 것이든 될 수 있는 Parser를 만들면 된다. 이를 위해 필요한 Parser에 대한 연산자를 정의하자.

In [29]:
(<|>) :: Parser a -> Parser a -> Parser a
p1 <|> p2 = \cs -> p1 cs ++ p2 cs

infixl 3 <|>

In [30]:
nat :: Parser Int
nat = zero <|> posint

int :: Parser Int
int = nat <|> negint

zero :: Parser Int
zero   = char '0' >>= \_ ->
         return 0

negint :: Parser Int
negint = char '-' >>= \_ ->
         posint   >>= \n ->
         return (-n)

----

In [31]:
token :: Parser a -> Parser a
token p = spaces >>= \_ ->
          p      >>= \v ->
          spaces >>= \_ ->
          return v

In [32]:
integer = token int
lowercase = token lower

In [33]:
integer " 123 "
integer " -123 "
integer " 0 "

[(1,"23 "),(12,"3 "),(123," "),(123,"")]

[(-1,"23 "),(-12,"3 "),(-123," "),(-123,"")]

[(0," "),(0,"")]

In [34]:
lowercase "   b  "
lowercase "   A  "

[('b',"  "),('b'," "),('b',"")]

[]

In [46]:
tok = token . char
tokDigit = token digit

애매한 문법
```
expr ::= expr '+' expr | expr '*' expr | digit | '(' expr ')'
digit ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

왼쪽재귀를 없앤 문법 (하지만 연산자 우선순위는 지정 안되어 있음)
```
expr ::= digit '+' expr | digit '*' expr | digit
digit ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

연산자 우선순위까지 지정하는 문법 (하지만 결합 방향은 우결합만 가능)
```
term   ::= digit   |  '(' expr ')'
factor ::= term    |  term   '*' factor
expr   ::= factor  |  factor '+' expr
```

연산자 결합 방향을 문법분석 후 처리하기 용이하도록 반복되는 부분을 Kleene 연산자로 묶어줌
```
term   ::= digit  |  '(' expr ')'
factor ::= term   ( '*' term   )*
expr   ::= factor ( '+' factor )*
```

In [70]:
-- 나무구조의 추상문법(abstract syntax)에는 연산자 우선순위나 괄호 등이 필요없다
-- 이미 나무구조로 어떤 식들이 어떻게 묶여 있는지 표현되기 때문
data Expr = Plus Expr Expr | Mult Expr Expr | Digit Char deriving Show

In [74]:
-- 재귀 하강 문법분석기 (recursive decendent parser)

digitExpr = tokDigit >>= \c ->
            return $ Digit c

term   = digitExpr <|> ( tok '(' >>= \_ ->
                         expr    >>= \e ->
                         tok ')' >>= \_ ->
                         return e )

factor = term                       >>= \e0 ->
         many ( tok '*' >>= \_ ->
                term  >>= \e ->
                return e          ) >>= \es ->
         return $ foldl Mult e0 es

expr   = factor                     >>= \e0 ->
         many ( tok '+' >>= \_ ->
                factor    >>= \e ->
                return e          ) >>= \es ->
         return $ foldl Plus e0 es

In [82]:
last $ term "1"
last $ term "(1+2)"

last $ factor "1"
last $ factor "1*2*3"

last $ expr "1"
last $ expr "1+2+3"
last $ expr "1+2*3"
last $ expr "1*2+3"

(Digit '1',"")

(Plus (Digit '1') (Digit '2'),"")

(Digit '1',"")

(Mult (Mult (Digit '1') (Digit '2')) (Digit '3'),"")

(Digit '1',"")

(Plus (Plus (Digit '1') (Digit '2')) (Digit '3'),"")

(Plus (Digit '1') (Mult (Digit '2') (Digit '3')),"")

(Plus (Mult (Digit '1') (Digit '2')) (Digit '3'),"")